In [68]:
import openrouteservice
from geopy.geocoders import Nominatim
import time
import polyline

ORS_API_KEY = "eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6ImExOWI5M2FjZTYwNjRjYzI4MTgwZmNmNmFjOWVkZWJlIiwiaCI6Im11cm11cjY0In0="
client = openrouteservice.Client(key=ORS_API_KEY)
geolocator = Nominatim(user_agent="rise-over-run", timeout=10)

# 1. Geocode addresses
def geocode_address(address):
    for _ in range(3):
        try:
            location = geolocator.geocode(address)
            if location:
                return [location.latitude, location.longitude]  # [lat, lon]
        except Exception as e:
            print("Retrying geocode:", e)
            time.sleep(1)
    raise ValueError(f"Could not geocode: {address}")

start_address = "111 S Jackson St, Seattle, WA 98104"
end_address = "423 Terry Ave, Seattle, WA 98104"

start_coords = geocode_address(start_address)
end_coords = geocode_address(end_address)

# 2. Get route
route = client.directions(
    coordinates=[start_coords, end_coords],
    profile='foot-walking',
    format='geojson'
)

# 3. Extract coordinates (as [lon, lat] list)
coords = route['features'][0]['geometry']['coordinates']
print("Sample route coordinates:", coords[:3])

# 4. Encode polyline
encoded_polyline = polyline.encode(coords)
print("Encoded polyline:", encoded_polyline)

# 5. Elevation API call
elevation = client.elevation_line(
    format_in='encodedpolyline',
    format_out='geojson',
    geometry=encoded_polyline
)

# 6. Print elevation data
print("Elevation response keys:", elevation.keys())
print("First few elevation points:")
for point in elevation['geometry']['coordinates'][:5]:
    print(point)  # [lon, lat, elevation]

ApiError: 404 ({'error': {'code': 2010, 'message': 'Could not find routable point within a radius of 350.0 meters of specified coordinate 0: 47.5990873 -122.3337386.'}, 'info': {'engine': {'build_date': '2025-06-06T15:39:25Z', 'graph_version': '2', 'graph_date': '2025-08-26T15:10:22Z', 'osm_date': '2025-08-18T00:00:00Z', 'version': '9.3.0'}, 'timestamp': 1757016400666}})